## experimenting...

In [1]:
from os.path import join

raw = join('data', 'raw')
interim = join('data', 'validation')

from src.training_env import reset_and_populate, populate

reset_and_populate(raw, interim, [400,100,0])

['data\\validation\\train\\n',
 'data\\validation\\validation\\n',
 'data\\validation\\test\\n',
 'data\\validation\\train\\o',
 'data\\validation\\validation\\o',
 'data\\validation\\test\\o',
 'data\\validation\\train\\x',
 'data\\validation\\validation\\x',
 'data\\validation\\test\\x']

## Data generator

In [2]:
import tensorflow as tf
import numpy as np
import pathlib

def create_generator(data_dir, batch_size=32, shuffle_buffer_size=1000):
        def get_label(file_path):
                parts = tf.strings.split(file_path, '\\')
                return parts[-2] == labels

        def decode_img(img):
                img = tf.image.decode_png(img, channels=1)
                img = tf.image.convert_image_dtype(img, tf.float32)
                return tf.image.resize(img, [32, 32])

        def process_path(file_path):
                label = get_label(file_path)
                img = tf.io.read_file(file_path)
                img = decode_img(img)
                return img, label

        data_dir = pathlib.Path(join(interim, data_dir))

        labels = np.array([item.name for item in data_dir.glob('*')])

        autotune = tf.data.experimental.AUTOTUNE

        dataset = (tf.data.Dataset.list_files(str(data_dir/'*/*'))
                .map(process_path, num_parallel_calls=autotune)
                .cache()
                .shuffle(shuffle_buffer_size)
                .repeat()
                .batch(batch_size)
                .prefetch(buffer_size=autotune))
        return dataset

train_generator = create_generator('train', 20)
validation_generator = create_generator('validation', 10)

In [3]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, RMSprop

def create_model(hp):
    model = models.Sequential()
    # Invariant Input
    model.add(layers.Flatten(input_shape=(32, 32, 1)))
    # Add n layers
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(layers.Dense(
            units=hp.Int('num_nodes', 16, 64, 4),
            activation='relu'))
    # Invariant Output
    model.add(layers.Dense(3, 'softmax'))
    # Select any Optimizer
    optimizers = {
        'sgd': SGD(
            lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01]),
            momentum=hp.Float('momentum', 0.5, 1, 0.1),
            nesterov=hp.Boolean('nesterov'))
    }
    # Compile Model
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizers[hp.Choice('optimizer', ['sgd'])],
        metrics=['acc'])

    return model

In [4]:
from datetime import datetime
from tensorboard.plugins.hparams import api
from kerastuner import RandomSearch
from tensorflow import summary
from tensorflow.keras.callbacks import TensorBoard

class customTuner(RandomSearch):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def run_trial(self, trial, *args, **kwargs):
        callbacks = kwargs.pop('callbacks', [])
        callbacks = self._deepcopy_callbacks(callbacks)
        trial_dir = self.get_trial_dir(trial.trial_id)
        callbacks.append(TensorBoard(trial_dir))
        kwargs['callbacks'] = callbacks
        super().run_trial(trial, *args, **kwargs)

    def on_trial_end(self, trial):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the hparams where the metrics of tensorboard are
        hparam_dir = join(trial_dir, trial.trial_id, 'execution0', 'train')
        hparams = trial.hyperparameters.values
        with summary.create_file_writer(hparam_dir).as_default():
            api.hparams(hparams, trial_id=trial.trial_id)
        print(datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
        print('Remaining Trials: ' + str(self.remaining_trials))
        super().on_trial_end(trial)

    def on_epoch_end(self, trial, model, epoch, logs):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the data where the metrics of tensorboard are
        hist_dir = join(trial_dir, trial.trial_id, 'execution0', 'train')
        with summary.create_file_writer(hist_dir).as_default():
            for layer in model.weights:
                summary.histogram(layer.name, data=layer, step=epoch)
        super().on_epoch_end(trial, model, epoch, logs)

In [5]:
from kerastuner import HyperParameters

hp=HyperParameters()
log_dir = 'trial_logs2'
timestamp = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

tuner = customTuner(
    create_model,
    hyperparameters=hp,
    objective='acc',
    max_trials=500,
    executions_per_trial=1,
    directory=log_dir,
    project_name=timestamp)

tuner.search_space_summary()

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [ EarlyStopping(monitor='loss', patience=3) ]

tuner.search(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    steps_per_epoch=30,
    validation_steps=30,
    verbose=0,
    callbacks=callbacks)

ValueError: Can not squeeze dim[1], expected a dimension of 1, got 3 for 'metrics/acc/Squeeze' (op: 'Squeeze') with input shapes: [?,3].

In [0]:
tuner.results_summary()